# Brisbane Burbs Best Business Battle
### Determining what type of business would be most successful in each Brisbane suburb.

This notebook attempts to determine what type of business might be best suitable for each suburb in Brisbane, Australia.

## Introduction / Business Problem

This notebook is aimed towards business entrepeneurs, or potential franchisees, who are looking to set up a new business in any of the suburbs around Brisbane, Australia. There are already a number of well established businesses in Brisbane, so this notebook attempts to analyze what type of businesses are most common in each suburb. The entrepreneur can then make a more informed desicion as to what type of business, per suburb, might be most successful.  

## Data

In order to solve this problem, two sets of data will be used. The first set of data contains the names of each suburb in Brisbane, the post code, the LGA, and the geospatial coordinates (longitude and latitude) of each suburb. This set of data is scraped using BeutifulSoup from the URL: https://www.geonames.org/postal-codes/AU/QLD/queensland.html. The data is 

The second set of data is from FourSquare, which contains location data such as venue name and venue category, based on the geospatial coordinates of the suburbs. The FourSquare API is used to retreive this data, using a free FourSquare developer account, which can be created here: https://developer.foursquare.com/

### Retreiving the first dataset, and cleaning/formatting it into a clean dataframe format.

Import required libraries to retreive the data

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup 

Scrape the URL to retreive the raw data used for the first set of data, and load it into a dataframe.

In [11]:
req = requests.get("https://www.geonames.org/postal-codes/AU/QLD/queensland.html")

soup = BeautifulSoup(req.content,'lxml')

table = soup.find_all('table')[2]

df = pd.read_html(str(table))

brisbane_df=pd.DataFrame(df[0])
brisbane_df

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
0,1.0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,NaN
1,NaN,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025
2,2.0,Brisbane City,4000,Australia,Queensland,NaN,NaN
3,NaN,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027
4,3.0,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,NaN
...,...,...,...,...,...,...,...
396,199.0,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT,NaN
397,NaN,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017
398,200.0,Robertson,4109,Australia,Queensland,UNDERWOOD CENT,NaN
399,NaN,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056


As can be seen above, the location data scraped show up on its own row (every other row). Let's clean this up so we have the latitude and longitude coordinates of each suburb in the same row.

The rows that contain 'NaN' values in column 'Unnamed: 0' has the coordinate values. Let's filter them out.

In [12]:
geo_data = brisbane_df['Unnamed: 0'].isna()
geo_data.head()

0    False
1     True
2    False
3     True
4    False
Name: Unnamed: 0, dtype: bool

Now we can create a dataframe that only contains the geospatial coordinates.

In [13]:
geo_df = brisbane_df[geo_data]
geo_df

,Unnamed: 0,Place,Code,Country,Admin1,Admin2,Admin3
1,NaN,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025,-27.462/153.025
3,NaN,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027,-27.468/153.027
5,NaN,-27.438/153.066,-27.438/153.066,-27.438/153.066,-27.438/153.066,-27.438/153.066,-27.438/153.066
7,NaN,-27.419/153.054,-27.419/153.054,-27.419/153.054,-27.419/153.054,-27.419/153.054,-27.419/153.054
9,NaN,-27.402/153.062,-27.402/153.062,-27.402/153.062,-27.402/153.062,-27.402/153.062,-27.402/153.062
...,...,...,...,...,...,...,...
393,NaN,-27.552/153.034,-27.552/153.034,-27.552/153.034,-27.552/153.034,-27.552/153.034,-27.552/153.034
395,NaN,-27.567/153.039,-27.567/153.039,-27.567/153.039,-27.567/153.039,-27.567/153.039,-27.567/153.039
397,NaN,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017,-27.565/153.017
399,NaN,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056,-27.565/153.056


Now let's remove (drop) the rows that contain 'NaN' values from our original dataframe. But first, let's drop the last column ('Admin3') as we don't need it.

In [14]:
brisbane_df.drop(['Admin3'], axis=1, inplace=True)
brisbane_df.dropna(how="all",inplace=True)

In [19]:
brisbane_df.dropna(how='any', inplace=True, axis=0 )
brisbane_df

,Place,Code,Country,Admin1,Admin2
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY
4,Hamilton,4007,Australia,Queensland,NORTHGATE CENT
6,Clayfield,4011,Australia,Queensland,NORTHGATE CENT
8,Nundah,4012,Australia,Queensland,NORTHGATE NORTH
10,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT
...,...,...,...,...,...
390,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT
392,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT
394,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT
396,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT


Let's also drop the first column ('Unnamed: 0') as we don't need that anymore either.

In [18]:
brisbane_df.drop(['Unnamed: 0'], axis=1, inplace=True)
brisbane_df.dropna(how="all",inplace=True)
brisbane_df

,Place,Code,Country,Admin1,Admin2
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY
4,Hamilton,4007,Australia,Queensland,NORTHGATE CENT
6,Clayfield,4011,Australia,Queensland,NORTHGATE CENT
8,Nundah,4012,Australia,Queensland,NORTHGATE NORTH
10,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT
...,...,...,...,...,...
390,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT
392,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT
394,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT
396,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT


Let's also reset the index of the dataframe.

In [20]:
brisbane_df.reset_index(drop=True, inplace=True)
brisbane_df

,Place,Code,Country,Admin1,Admin2
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT
...,...,...,...,...,...
191,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT
192,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT
193,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT
194,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT


Since the latitude and longitude values are in the same column, let's split them into separate columns and label them 'longitude' and 'latitude' respectively.

In [22]:
geo_df = geo_df.Place.str.split(pat='/', expand=True)
geo_df

,0,1
1,-27.462,153.025
3,-27.468,153.027
5,-27.438,153.066
7,-27.419,153.054
9,-27.402,153.062
...,...,...
393,-27.552,153.034
395,-27.567,153.039
397,-27.565,153.017
399,-27.565,153.056


In [23]:
geo_df.rename(columns={0:'latitude',
                      1:'longitude'}, inplace=True)
geo_df.columns

Index(['latitude', 'longitude'], dtype='object')

Let's alos reset the index.

In [24]:
geo_df.reset_index(drop=True, inplace=True)
geo_df

,latitude,longitude
0,-27.462,153.025
1,-27.468,153.027
2,-27.438,153.066
3,-27.419,153.054
4,-27.402,153.062
...,...,...
196,-27.552,153.034
197,-27.567,153.039
198,-27.565,153.017
199,-27.565,153.056


Finally, let's combine the two dataframes into a final, merged dataframe that contains both the suburb names, codes, etc. as well as the geospatial coordinates.

In [25]:
brisbane_geo_df = pd.concat([brisbane_df, geo_df], axis=1, sort=False)
brisbane_geo_df

,Place,Code,Country,Admin1,Admin2,latitude,longitude
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,-27.462,153.025
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,-27.468,153.027
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT,-27.438,153.066
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH,-27.419,153.054
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT,-27.402,153.062
...,...,...,...,...,...,...,...
196,NaN,NaN,NaN,NaN,NaN,-27.552,153.034
197,NaN,NaN,NaN,NaN,NaN,-27.567,153.039
198,NaN,NaN,NaN,NaN,NaN,-27.565,153.017
199,NaN,NaN,NaN,NaN,NaN,-27.565,153.056


In [29]:
brisbane_geo_df.shape

(196, 7)

Let's also label the columns more appropriately.

In [30]:
brisbane_geo_df.rename(columns={'Place':'Suburb',
                                'Admin1':'State',
                                'Admin2':'LGA'
                               }, inplace=True)
brisbane_geo_df.columns

Index(['Suburb', 'Code', 'Country', 'State', 'LGA', 'latitude', 'longitude'], dtype='object')

And also drop any remaining rows with 'NaN' values. Finally we have our first dataset in a dataframe format.

In [28]:
brisbane_geo_df.dropna(inplace=True)
brisbane_geo_df

,Suburb,Code,Country,State,LGA,latitude,longitude
0,Spring Hill,4000,Australia,Queensland,BRIS CITY CNTRY,-27.462,153.025
1,Hamilton,4007,Australia,Queensland,NORTHGATE CENT,-27.468,153.027
2,Clayfield,4011,Australia,Queensland,NORTHGATE CENT,-27.438,153.066
3,Nundah,4012,Australia,Queensland,NORTHGATE NORTH,-27.419,153.054
4,Bracken Ridge,4017,Australia,Queensland,NORTHGATE CENT,-27.402,153.062
...,...,...,...,...,...,...,...
191,Rocklea,4106,Australia,Queensland,UNDERWOOD CENT,-27.507,153.024
192,Salisbury,4107,Australia,Queensland,UNDERWOOD CENT,-27.515,153.013
193,Coopers Plains,4108,Australia,Queensland,UNDERWOOD CENT,-27.525,153.016
194,Archerfield,4108,Australia,Queensland,UNDERWOOD CENT,-27.525,153.004


In [31]:
brisbane_geo_df.shape

(196, 7)

### Retreiving the second dataset from FourSquare, via their API.

Importing the required libraries.

In [32]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import json # library to handle JSON files

from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe


Exploring the Brisbane, Australia location.

In [33]:
address = 'Brisbane, Australia'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Brisbane, Australia are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Brisbane, Australia are -27.4689682, 153.0234991.


Defining the FourSquare API credentials.

In [35]:
CLIENT_ID = 'QKAHEWSRIS0OJH5TAJUAZPYTUR3AAZAM1UFYOY2FBHKH0G1P' # your Foursquare ID
CLIENT_SECRET = 'FUN0ESQLCT52Z5PKNZV3G1MCZ3KKCNJ20KWWWTFJ2V52GF3C' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: QKAHEWSRIS0OJH5TAJUAZPYTUR3AAZAM1UFYOY2FBHKH0G1P
CLIENT_SECRET:FUN0ESQLCT52Z5PKNZV3G1MCZ3KKCNJ20KWWWTFJ2V52GF3C


Let's explore the first suburb in the dataframe.

In [36]:
brisbane_geo_df.loc[0, 'Suburb']

'Spring Hill'

In [37]:
suburb_latitude = brisbane_geo_df.loc[0, 'latitude'] # neighborhood latitude value
suburb_longitude = brisbane_geo_df.loc[0, 'longitude'] # neighborhood longitude value

suburb_name = brisbane_geo_df.loc[0, 'Suburb'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(suburb_name, 
                                                               suburb_latitude, 
                                                               suburb_longitude))

Latitude and longitude values of Spring Hill are -27.462, 153.025.


Now let's define the variables used to retreive data from the FourSquare API.

In [38]:
LIMIT = 100 # limit of number of venues returned by Foursquare API
radius = 500 # define radius

url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    suburb_latitude, 
    suburb_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=QKAHEWSRIS0OJH5TAJUAZPYTUR3AAZAM1UFYOY2FBHKH0G1P&client_secret=FUN0ESQLCT52Z5PKNZV3G1MCZ3KKCNJ20KWWWTFJ2V52GF3C&v=20180605&ll=-27.462,153.025&radius=500&limit=100'

And retreive the results.

In [39]:
results = requests.get(url).json()

In [40]:
results

{'meta': {'code': 200, 'requestId': '5ecf1798949393001c555725'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Spring Hill',
  'headerFullLocation': 'Spring Hill, Brisbane',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 36,
  'suggestedBounds': {'ne': {'lat': -27.457499995499994,
    'lng': 153.030062004985},
   'sw': {'lat': -27.466500004500006, 'lng': 153.019937995015}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '527e183911d2a639621d2310',
       'name': 'jak + hill',
       'location': {'address': '40 Astor Tce',
        'lat': -27.4636262353871,
        'lng': 153.0256167938703,
        'labeledLatLngs': [{'label': 'display',
          'lat': -27.46362

Next, let's extract the venue category for each venue.

In [41]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [42]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head(10)

/home/joakim/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  This is separate from the ipykernel package so we can avoid doing imports until


,name,categories,lat,lng
0,jak + hill,Café,-27.463626,153.025617
1,Punthill Apartment Hotels,Hotel,-27.463598,153.025617
2,The Alliance Hotel,Gastropub,-27.461272,153.026370
3,Bishamon Japanese Restaurant,Japanese Restaurant,-27.460428,153.021727
4,Sisco Bcl,Café,-27.460456,153.021788
5,Domino's Pizza,Pizza Place,-27.461136,153.024413
6,WickEd Corner Cafe,Café,-27.465096,153.024788
7,Spring Hill Deli,Deli / Bodega,-27.460371,153.021384
8,St Paul's Tavern,Pub,-27.461825,153.026383
9,Toobirds Bistro & Bar | Expresso & Eats,Bistro,-27.465776,153.027283


In [43]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

36 venues were returned by Foursquare.


Let's also get the nearby venues in a radius of 500 meters.

In [44]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Suburb', 
                  'Suburb Latitude', 
                  'Suburb Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [45]:
brisbane_venues = getNearbyVenues(names=brisbane_geo_df['Suburb'],
                                   latitudes=brisbane_geo_df['latitude'],
                                   longitudes=brisbane_geo_df['longitude']
                                  )

Spring Hill
Hamilton
Clayfield
Nundah
Bracken Ridge
Redcliffe
Kedron
Chermside
Aspley
Albany Creek
Kelvin Grove
Ashgrove
The Gap
Paddington
Toowong
St Lucia
Taringa
Indooroopilly
Inala
Forest Lake
West End
Woolloongabba
Annerley
Moorooka
Sunnybank Hills
Runcorn
Eight Mile Plains
Woodridge
Calamvale
Greenslopes
Upper Mount Gravatt
Mount Gravatt East
Slacks Creek
Coorparoo
Camp Hill
Carina
Carindale
Capalaba
Birkdale
Alexandra Hills
Thornlands
Redland Bay
Victoria Point
Wynnum
Wynnum West
Russell Island
Eagleby
Ormeau
Coomera
Pimpama
Upper Coomera
Oxenford
Nerang
Pacific Pines
Carrara
Hope Island
Helensvale
Mudgeeraba
Ashmore
Arundel
Labrador
Southport
Runaway Bay
Coombabah
Main Beach
Surfers Paradise
Mermaid Waters
Mermaid Beach
Broadbeach
Burleigh Waters
Miami
Burleigh Heads
Palm Beach
Coolangatta
Robina
Varsity Lakes
Jimboomba
Springfield Lakes
Redbank Plains
Warwick
Dalby
Kallangur
Narangba
Burpengary
Morayfield
Bongaree
Deception Bay
Caboolture
Buderim
Mooloolaba
Maroochydore
Nambou

In [46]:
print(brisbane_venues.shape)
brisbane_venues.head(5)

(1589, 7)


,Suburb,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Spring Hill,-27.462,153.025,jak + hill,-27.463626,153.025617,Café
1,Spring Hill,-27.462,153.025,Punthill Apartment Hotels,-27.463598,153.025617,Hotel
2,Spring Hill,-27.462,153.025,The Alliance Hotel,-27.461272,153.026370,Gastropub
3,Spring Hill,-27.462,153.025,Bishamon Japanese Restaurant,-27.460428,153.021727,Japanese Restaurant
4,Spring Hill,-27.462,153.025,Sisco Bcl,-27.460456,153.021788,Café


Next we'll count the number of venues for each suburb.

In [47]:
brisbane_venues_group_count = brisbane_venues.groupby('Suburb').count()
brisbane_venues_group_count

,Suburb Latitude,Suburb Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Suburb,,,,,,
Albany Creek,13,13,13,13,13,13
Albion,12,12,12,12,12,12
Alexandra Hills,3,3,3,3,3,3
Annerley,19,19,19,19,19,19
Arana Hills,7,7,7,7,7,7
...,...,...,...,...,...,...
Wynnum,2,2,2,2,2,2
Wynnum West,6,6,6,6,6,6
Yeerongpilly,4,4,4,4,4,4


In [48]:
print('There are {} uniques categories.'.format(len(brisbane_venues['Venue Category'].unique())))

There are 228 uniques categories.
